<a href="https://colab.research.google.com/github/dwwatson1/coronavirus_pandemic_playbook/blob/michael_work/cdc_main_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

In [49]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").getOrCreate()

In [53]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://cdc-main-database.s3.amazonaws.com/COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv"), header=True)

In [54]:
df.show()

+----------+---------+---------------+----------+----------------+--------------+------+-------+---------+-------------------------------+-------------------+--------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|case_month|res_state|state_fips_code|res_county|county_fips_code|     age_group|   sex|   race|ethnicity|case_positive_specimen_interval|case_onset_interval|             process|exposure_yn|      current_status|symptom_status|hosp_yn| icu_yn|death_yn|underlying_conditions_yn|
+----------+---------+---------------+----------+----------------+--------------+------+-------+---------+-------------------------------+-------------------+--------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|   2020-08|       KY|             21|   GRAYSON|           21085|  0 - 17 years|    NA|     NA|       NA|                           null|                  0|        

In [59]:
# Drop unnecessary columns
df = df.drop("case_positive_specimen_interval", "case_onset_interval")

In [60]:
df.show()

+----------+---------+---------------+----------+----------------+--------------+------+-------+---------+--------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|case_month|res_state|state_fips_code|res_county|county_fips_code|     age_group|   sex|   race|ethnicity|             process|exposure_yn|      current_status|symptom_status|hosp_yn| icu_yn|death_yn|underlying_conditions_yn|
+----------+---------+---------------+----------+----------------+--------------+------+-------+---------+--------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|   2020-08|       KY|             21|   GRAYSON|           21085|  0 - 17 years|    NA|     NA|       NA|             Missing|    Missing|Laboratory-confir...|   Symptomatic|     No|Missing|      No|                    null|
|   2021-01|       NC|             37|    MARTIN|           37117|  0 - 17 years|    NA|     NA|

In [61]:
# Drop all null values
df = df.na.drop(how="any")

In [62]:
df.show()

+----------+---------+---------------+----------+----------------+--------------+------+-------+-------------------+--------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|case_month|res_state|state_fips_code|res_county|county_fips_code|     age_group|   sex|   race|          ethnicity|             process|exposure_yn|      current_status|symptom_status|hosp_yn| icu_yn|death_yn|underlying_conditions_yn|
+----------+---------+---------------+----------+----------------+--------------+------+-------+-------------------+--------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|   2021-03|       VT|             50|  FRANKLIN|           50011|18 to 49 years|    NA|     NA|                 NA|             Missing|        Yes|Laboratory-confir...|   Symptomatic|     No|Missing|      No|                     Yes|
|   2020-08|       AR|             05|    LONOKE|       

In [67]:
# Drop rows with NA
df = df.filter((df.sex != 'NA') & (df.race != 'NA') & (df.ethnicity != 'NA') & (df.death_yn != 'NA'))

In [68]:
df.show()

+----------+---------+---------------+------------+----------------+--------------+------+-------+-------------------+--------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|case_month|res_state|state_fips_code|  res_county|county_fips_code|     age_group|   sex|   race|          ethnicity|             process|exposure_yn|      current_status|symptom_status|hosp_yn| icu_yn|death_yn|underlying_conditions_yn|
+----------+---------+---------------+------------+----------------+--------------+------+-------+-------------------+--------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|   2020-11|       WA|             53|   SNOHOMISH|           53061|18 to 49 years|Female|Missing|    Hispanic/Latino|             Missing|    Missing|Laboratory-confir...|   Symptomatic|     No|Missing| Missing|                     Yes|
|   2020-07|       MD|             24|  DORCHEST

In [70]:
# Drop unnecessary columns
df = df.drop("process")

In [71]:
df.show()

+----------+---------+---------------+------------+----------------+--------------+------+-------+-------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|case_month|res_state|state_fips_code|  res_county|county_fips_code|     age_group|   sex|   race|          ethnicity|exposure_yn|      current_status|symptom_status|hosp_yn| icu_yn|death_yn|underlying_conditions_yn|
+----------+---------+---------------+------------+----------------+--------------+------+-------+-------------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|   2020-11|       WA|             53|   SNOHOMISH|           53061|18 to 49 years|Female|Missing|    Hispanic/Latino|    Missing|Laboratory-confir...|   Symptomatic|     No|Missing| Missing|                     Yes|
|   2020-07|       MD|             24|  DORCHESTER|           24019|18 to 49 years|Female|  White|    Hispanic/Latino|    Missing|La

In [73]:
# Drop rows with Missing
df = df.filter((df.race != 'Missing') & (df.ethnicity != 'Missing') & (df.exposure_yn != 'Missing') & (df.hosp_yn != 'Missing') & (df.icu_yn != 'Missing') & (df.death_yn != 'Missing'))

In [74]:
df.show()

+----------+---------+---------------+----------+----------------+--------------+------+--------------+---------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|case_month|res_state|state_fips_code|res_county|county_fips_code|     age_group|   sex|          race|      ethnicity|exposure_yn|      current_status|symptom_status|hosp_yn| icu_yn|death_yn|underlying_conditions_yn|
+----------+---------+---------------+----------+----------------+--------------+------+--------------+---------------+-----------+--------------------+--------------+-------+-------+--------+------------------------+
|   2020-04|       PA|             42|    LEHIGH|           42077|18 to 49 years|Female|         Black|Hispanic/Latino|        Yes|Laboratory-confir...|   Symptomatic|     No|     No|      No|                     Yes|
|   2020-07|       NC|             37|      NASH|           37127|18 to 49 years|Female|         White|Hispanic/Latino|    Unkno

In [78]:
# Drop rows with Unknown
df = df.filter((df.race != 'Unknown') & (df.ethnicity != 'Unknown') & (df.exposure_yn != 'Unknown') & (df.symptom_status != 'Unknown') & (df.hosp_yn != 'Unknown') & (df.icu_yn != 'Unknown') & (df.death_yn != 'Unknown'))

In [79]:
df.show()

+----------+---------+---------------+----------+----------------+--------------+------+--------------+---------------+-----------+--------------------+--------------+-------+------+--------+------------------------+
|case_month|res_state|state_fips_code|res_county|county_fips_code|     age_group|   sex|          race|      ethnicity|exposure_yn|      current_status|symptom_status|hosp_yn|icu_yn|death_yn|underlying_conditions_yn|
+----------+---------+---------------+----------+----------------+--------------+------+--------------+---------------+-----------+--------------------+--------------+-------+------+--------+------------------------+
|   2020-04|       PA|             42|    LEHIGH|           42077|18 to 49 years|Female|         Black|Hispanic/Latino|        Yes|Laboratory-confir...|   Symptomatic|     No|    No|      No|                     Yes|
|   2020-11|       PA|             42|   LUZERNE|           42079|18 to 49 years|Female|         White|Hispanic/Latino|        Yes|L

In [82]:
# Filter dates between March 2020 and December 2020
df2 = df.filter((df.case_month >= '2020-03') & (df.case_month <= '2020-12'))

In [83]:
df2.show()

+----------+---------+---------------+----------+----------------+--------------+------+--------------+---------------+-----------+--------------------+--------------+-------+------+--------+------------------------+
|case_month|res_state|state_fips_code|res_county|county_fips_code|     age_group|   sex|          race|      ethnicity|exposure_yn|      current_status|symptom_status|hosp_yn|icu_yn|death_yn|underlying_conditions_yn|
+----------+---------+---------------+----------+----------------+--------------+------+--------------+---------------+-----------+--------------------+--------------+-------+------+--------+------------------------+
|   2020-04|       PA|             42|    LEHIGH|           42077|18 to 49 years|Female|         Black|Hispanic/Latino|        Yes|Laboratory-confir...|   Symptomatic|     No|    No|      No|                     Yes|
|   2020-11|       PA|             42|   LUZERNE|           42079|18 to 49 years|Female|         White|Hispanic/Latino|        Yes|L

In [84]:
df2.count()

141944

In [109]:
df2.toPandas().to_csv('cdc_database_cleaned.csv')